In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timedelta
import json

In [2]:
cd data


c:\WORK\Oraichain\data


### Using pd to process data in csv format

In [6]:
df1 =pd.read_csv('closed.csv')
df1=df1.dropna(axis=1)
df2 = pd.read_csv('opening.csv')


- take_profit_pct = (take_profit/entry_price -1) * leverage (BUY)
- stop_loss_pct = (stop_loss/entry_price - 1)* leverage (BUY)
- take_profit_take = tak_profit_pct * margin_amount
- stop_loss_give = stop_loss_pct*margin_amount

In [11]:
df2

,pair,direction,entry_price,take_profit,stop_loss,block_time,order_id,bidder_addr,status,leverage,...,user_type,pnl,funding_payment,hash,ask_amount,offer_amount,stop_loss_pct,take_profit_pct,take_profit_take,stop_loss_give
0,ORAI_USDC,SELL,4.104134,3.570000,NaN,1701280171,397,orai1mvdnt277449t26f6g38epa0ugu5wmallgdaya7,OPENING,1.000000,...,USER,0,0,NaN,0,0,-1.000000,0.130145,2.438894,-18.739767
1,ORAI_USDC,SELL,4.044163,3.639746,NaN,1701313793,450,orai1ax88yuwrmapcmrrfm3r7c8xugphjgpgsuptsux,OPENING,1.000000,...,USER,0,0,NaN,0,0,-1.000000,0.100000,0.124283,-1.242828
2,ORAI_USDC,SELL,4.133886,4.017036,NaN,1701411890,528,orai18vyspza5ndpg5wss0u7mqs7g50zekjrjvc2dys,OPENING,1.908257,...,USER,0,0,NaN,0,0,-1.000000,0.053940,29.644085,-549.580000
3,ORAI_USDC,SELL,4.174882,4.035719,NaN,1701414166,532,orai18vyspza5ndpg5wss0u7mqs7g50zekjrjvc2dys,OPENING,1.908257,...,USER,0,0,NaN,0,0,-1.000000,0.063609,34.958067,-549.580000
4,ORAI_USDC,SELL,4.476720,4.327496,NaN,1701421454,566,orai18vyspza5ndpg5wss0u7mqs7g50zekjrjvc2dys,OPENING,3.000000,...,USER,0,0,NaN,0,0,-1.000000,0.100000,39.952000,-399.520000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,INJ_USDC,SELL,17.709588,15.938629,NaN,1701679805,883,orai12zyu8w93h0q2lcnt50g3fn0w3yqnhy4fvawaqz,OPENING,1.000000,...,USER,0,0,NaN,0,0,-1.000000,0.100000,0.099960,-0.999600
66,INJ_USDC,BUY,16.593808,16.870436,11.615710,1701688926,905,orai1nza67tu4pv6uueqmynytqzelejpykr5d3tkqnd,OPENING,3.000000,...,BOT,0,0,NaN,0,0,-0.899992,0.050012,163.001481,-2933.316037
67,INJ_USDC,BUY,16.692873,16.971142,11.685048,1701689000,906,orai1nza67tu4pv6uueqmynytqzelejpykr5d3tkqnd,OPENING,3.000000,...,BOT,0,0,NaN,0,0,-0.899993,0.050010,135.552854,-2439.455970
68,INJ_USDC,BUY,16.786510,17.066344,11.750597,1701689150,907,orai1nza67tu4pv6uueqmynytqzelejpykr5d3tkqnd,OPENING,3.000000,...,BOT,0,0,NaN,0,0,-0.899993,0.050011,145.875488,-2625.185890


In [ ]:
def pre_process_open(df2):
    df2['stop_loss_pct'] = df2.apply(lambda x: (x['stop_loss'] / x['entry_price'] - 1)*x['leverage'] if x['direction'] == "BUY" 
    else (1 - x['stop_loss'] / x['entry_price'])*x['leverage']
    if not np.isnan(x['stop_loss']) else -1, axis=1)
    df2['take_profit_pct'] = df2.apply(lambda x: (x['take_profit']/x['entry_price']-1)*x['leverage'] if x['direction'] == "BUY" 
        else (1 - x['take_profit'] / x['entry_price'])*x['leverage'], axis=1)
    df2['take_profit_take'] = df2['margin_amount'] * df2['take_profit_pct']
    df2['stop_loss_give'] = df2['margin_amount'] * df2['stop_loss_pct']



In [16]:
df = df2[['direction','margin_amount','stop_loss_give','take_profit_take']]

In [18]:
df.groupby('direction').sum()

,margin_amount,stop_loss_give,take_profit_take
direction,,,
BUY,97289.967831,-83175.941525,5846.123137
SELL,17664.161166,-5668.327405,6186.944724


In [4]:
df2

,pair,direction,entry_price,take_profit,block_time,order_id,bidder_addr,status,leverage,margin_amount,fee,time,user_type,pnl,funding_payment,ask_amount,offer_amount
0,ORAI_USDC,SELL,4.104134,3.570000,1701280171,397,orai1mvdnt277449t26f6g38epa0ugu5wmallgdaya7,OPENING,1.000000,18.739767,0.007498,2023-12-04 18:30:53,USER,0,0,0,0
1,ORAI_USDC,SELL,4.044163,3.639746,1701313793,450,orai1ax88yuwrmapcmrrfm3r7c8xugphjgpgsuptsux,OPENING,1.000000,1.242828,0.000496,2023-12-04 18:30:53,USER,0,0,0,0
2,ORAI_USDC,SELL,4.133886,4.017036,1701411890,528,orai18vyspza5ndpg5wss0u7mqs7g50zekjrjvc2dys,OPENING,1.908257,549.580000,0.420000,2023-12-04 18:30:53,USER,0,0,0,0
3,ORAI_USDC,SELL,4.174882,4.035719,1701414166,532,orai18vyspza5ndpg5wss0u7mqs7g50zekjrjvc2dys,OPENING,1.908257,549.580000,0.420000,2023-12-04 18:30:53,USER,0,0,0,0
4,ORAI_USDC,SELL,4.476720,4.327496,1701421454,566,orai18vyspza5ndpg5wss0u7mqs7g50zekjrjvc2dys,OPENING,3.000000,399.520000,0.480000,2023-12-04 18:30:53,USER,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,INJ_USDC,SELL,17.709588,15.938629,1701679805,883,orai12zyu8w93h0q2lcnt50g3fn0w3yqnhy4fvawaqz,OPENING,1.000000,0.999600,0.000400,2023-12-04 18:30:53,USER,0,0,0,0
66,INJ_USDC,BUY,16.593808,16.870436,1701688926,905,orai1nza67tu4pv6uueqmynytqzelejpykr5d3tkqnd,OPENING,3.000000,3259.269110,3.915820,2023-12-04 18:30:53,BOT,0,0,0,0
67,INJ_USDC,BUY,16.692873,16.971142,1701689000,906,orai1nza67tu4pv6uueqmynytqzelejpykr5d3tkqnd,OPENING,3.000000,2710.526606,3.256538,2023-12-04 18:30:53,BOT,0,0,0,0
68,INJ_USDC,BUY,16.786510,17.066344,1701689150,907,orai1nza67tu4pv6uueqmynytqzelejpykr5d3tkqnd,OPENING,3.000000,2916.896380,3.504480,2023-12-04 18:30:53,BOT,0,0,0,0


In [55]:
type(df1.time)
current_time = datetime.now()

time_ranges = {
    '1h': current_time - timedelta(hours=1),
    '4h': current_time - timedelta(hours=4),
    '24h': current_time - timedelta(hours=24),
    '7d': current_time - timedelta(days=7),
    '30d': current_time - timedelta(days=30),
    '90d': current_time - timedelta(days=90)
}
# time_24_hours_ago = current_time - timedelta(hours=24)
label = '1h'
if label in time_ranges:
    selected_range = time_ranges[label]
    print(f"{label} time range starts from: {selected_range}")
else:
    print(f"{label} is not a valid time range label.")
df1.time = pd.to_datetime(df1.time)
filtered_data = df1[
        (df1['time'] >= selected_range) & (df1['time'] <= current_time)
]

1h time range starts from: 2023-12-05 14:54:05.742709


In [56]:
filtered_data

,user_type,pair,hash,height,time,order_id,status,direction,bidder_addr,offer_amount,...,stop_loss,pnl,fee,funding_payment,volume,close_price,clost_pct,close_pct,lose_amount,win_amount


In [53]:
filtered_time

,user_type,pair,hash,height,time,order_id,status,direction,bidder_addr,offer_amount,...,stop_loss,pnl,fee,funding_payment,volume,close_price,clost_pct,close_pct,lose_amount,win_amount
787,USER,ORAI_USDC,636C5677AACFDEB48C18C13F2367D01EBFC444550023F6...,14685433,2023-12-04 16:02:21,881,Close,SELL,orai1hvhkk0jrdu6dz2qwum2v2arc8rtucha79jkfdn,1037.995110,...,0.000000,-0.000006,0.415696,0.933525,1037.995104,5.141465,-4.081164e-07,-4.081164e-07,-0.000141,0.000000
788,BOT,INJ_USDC,1FDAFFA47545B50D6BFE17CF78817DCE18C5E01A50DF80...,14685522,2023-12-04 16:10:54,865,Close,BUY,orai1nza67tu4pv6uueqmynytqzelejpykr5d3tkqnd,10584.651350,...,12.266853,75.216553,4.208824,30.496780,10509.434796,17.649427,2.147122e-02,2.147122e-02,0.000000,75.216789
789,USER,ORAI_USDC,B441FEB8C971766B499C836482FC1FDA62817CC06911BC...,14685878,2023-12-04 16:44:51,798,Close,SELL,orai1xsx9ezzynw4thhgh7ghpffvc28e0d4wswedjuc,205.318452,...,0.000000,6.460185,0.084812,1.819666,211.778637,5.157259,9.151282e-02,9.151282e-02,0.000000,6.460153
790,USER,ORAI_USDC,AA3247D4EBB0976C01EE080BAD83A736D1DF232502069B...,14685869,2023-12-04 16:43:59,858,Close,SELL,orai1uesy4qnqathqlxalakh9r5slvjlvq6appzdhjr,1988.300855,...,0.000000,61.236745,0.820800,5.074771,2049.537600,5.150617,8.963478e-02,8.963478e-02,0.000000,61.236617
791,USER,ORAI_USDC,E6EBCD21DB6B4585979339D404714715F421FD1D9401CE...,14685913,2023-12-04 16:48:10,869,Close,SELL,orai1gm7ptpux85zu2e9p4vpnwchpueh4mv8qyerxtu,2884.348008,...,0.000000,61.068657,1.179582,5.677570,2945.416665,5.166615,6.220015e-02,6.220015e-02,0.000000,61.068457
792,BOT,INJ_USDC,37F1FBAA943E637EF61C62EE9EBA1EFA89146B92E42D74...,14686005,2023-12-04 16:56:58,866,Close,BUY,orai1nza67tu4pv6uueqmynytqzelejpykr5d3tkqnd,7792.328959,...,12.339656,3.845671,3.119136,22.467627,7788.483288,17.636731,1.481313e-03,1.481313e-03,0.000000,3.845726
793,BOT,INJ_USDC,37F1FBAA943E637EF61C62EE9EBA1EFA89146B92E42D74...,14686005,2023-12-04 16:56:58,867,Close,BUY,orai1nza67tu4pv6uueqmynytqzelejpykr5d3tkqnd,10980.845890,...,12.415064,-128.881648,4.449230,31.853881,11109.727542,17.529982,-3.480222e-02,-3.480222e-02,-128.881064,0.000000
794,BOT,INJ_USDC,8752889D167C0F71BF239F7AE7E3321939C39FCB7ECBB5...,14686019,2023-12-04 16:58:17,868,Close,BUY,orai1nza67tu4pv6uueqmynytqzelejpykr5d3tkqnd,8968.113602,...,12.421717,-103.813474,3.633130,25.997134,9071.927076,17.542184,-3.433006e-02,-3.433006e-02,-103.813258,0.000000
795,USER,ORAI_USDC,236DC646488D963760676C86F7B0FA8AE84745BDBB56DA...,14686209,2023-12-04 17:16:24,589,Close,BUY,orai1ax88yuwrmapcmrrfm3r7c8xugphjgpgsuptsux,6.330486,...,0.000000,0.976809,0.002144,0.026317,5.353677,5.191412,5.473672e-01,5.473672e-01,0.000000,0.976809
796,BOT,ORAI_USDC,E48D0D63C0F0AE9561136A9DBAD91B0D47B0A6C0C238C3...,14686099,2023-12-04 17:05:55,705,TP,BUY,orai15vc7z4zyyam999zp6rwealm06lvuy9ykumwlpl,4846.882216,...,3.500777,154.173844,1.879338,-10.770222,4698.346386,5.165400,9.856230e-02,9.856230e-02,0.000000,154.359935


In [49]:
df1.time

0     2023-11-08 10:20:57
1     2023-11-08 10:19:09
2     2023-11-21 15:23:54
3     2023-11-21 17:40:32
4     2023-11-21 18:08:23
              ...        
817   2023-12-04 18:15:31
818   2023-12-04 18:18:17
819   2023-12-04 18:19:02
820   2023-12-04 18:19:02
821   2023-12-04 18:20:40
Name: time, Length: 822, dtype: datetime64[ns]

In [39]:
#add new colume to df
df1=df1.dropna(axis=1)
df1['volume'] = df1['leverage'] * df1['margin_amount']
df1['close_price'] =df1['offer_amount']/df1['ask_amount']
df1['close_pct'] = df1.apply(lambda x: (x['close_price'] / x['entry_price'] - 1)*x['leverage'] if x['direction'] == "BUY" else (1 - x['close_price'] / x['entry_price'])*x['leverage'], axis=1)
df1['lose_amount'] =df1.apply(lambda x:(x['close_pct']*x['margin_amount']) if x['close_pct'] <0 else 0, axis = 1)
df1['win_amount'] = df1.apply(lambda x: (x['close_pct']*x['margin_amount']) if x['close_pct'] >=0 else 0, axis = 1)


Index(['user_type', 'pair', 'hash', 'height', 'time', 'order_id', 'status',
       'direction', 'bidder_addr', 'offer_amount', 'ask_amount', 'leverage',
       'margin_amount', 'entry_price', 'take_profit', 'stop_loss', 'pnl',
       'fee', 'funding_payment', 'volume', 'close_price', 'clost_pct',
       'close_pct'],
      dtype='object')


In [31]:
print(df1.sort_values(by='clost_pct',ascending=False))
print(df1[df1['hash']=='130B795E1939A5C47F55EC225AEB876D62868F85ED2752E54F5FF432E0856725'])

    user_type       pair                                               hash   
735      USER  ORAI_USDC  130B795E1939A5C47F55EC225AEB876D62868F85ED2752...  \
757      USER  ORAI_USDC  ACE27955E29F9A1C088002FCDA89B231D744A02D0540A6...   
543      USER  ORAI_USDC  988420BD7BC7B7C79ED7995E86C75CAB84C4A414065E5C...   
558      USER  ORAI_USDC  630A3A8E1C6250B6E65F36F4BF9388BCD97152FC16B47C...   
795      USER  ORAI_USDC  236DC646488D963760676C86F7B0FA8AE84745BDBB56DA...   
..        ...        ...                                                ...   
677      USER  ORAI_USDC  3B09D4664F6DB9A02779E25A2AE7AEE3C36A065069564C...   
714      USER  ORAI_USDC  412DB9DB9CF34BCC70C4F0E74ED65425BD048010CE10A7...   
712      USER  ORAI_USDC  CBEFFE12FA5FF834E2051C0CA7A429EC86BF61275F1856...   
711      USER  ORAI_USDC  EF6E419D583E7EF16BA0BE911917A12711411B6B1C964F...   
700      USER  ORAI_USDC  27B91AA4500F6A7DCC281ABFAB42CF95498AF558919E67...   

       height                 time  order_id     st

In [6]:
df2.head(1)

,close_price,close_pct,lose_amount,win_amount,volume,user_type,pair,hash,height,time,...,offer_amount,ask_amount,leverage,margin_amount,entry_price,take_profit,stop_loss,pnl,fee,funding_payment
0,3.910011222,-0.00007%,-$0.000001,$0.000000,$3.00,USER,ORAI_USDC,AA677938E124AE43C10D99B1B205D3717B11AAC2B6555B...,14291688.0,2023-11-08 10:20:57,...,2.996398,0.76634,1.499099,1.9988,3.910013,4.040346,0.0,-0.000002,0.0012,0.0


In [41]:
df = df1[['user_type','lose_amount','win_amount']]
#remove NAN
df = df.dropna()
#remove dolar sign if have
# df['lose_amount'] = df['lose_amount'].str.replace('$','')
# df['win_amount'] = df['win_amount'].str.replace('$','')
# #make type to numeric
# df['lose_amount'] = pd.to_numeric(df['lose_amount'], errors='coerce')
# df['win_amount'] = pd.to_numeric(df['win_amount'], errors='coerce')
# df['lose_amount'] = df['lose_amount'].astype(float)
# df['win_amount'] = df['win_amount'].astype(float)
#group by user type
df = df.groupby('user_type').sum()


In [43]:
df

,lose_amount,win_amount
user_type,,
BOT,-17674.090548,34682.101329
USER,-4312.135463,10220.191347


In [11]:
#get win amount of user and bot


win_amount_user =  df.loc['USER']['win_amount']
    # loss_amount_user = get_win_loss(closed)['lose_amount'][['user_type'== "USER"]]
    # win_amount_bot = get_win_loss(closed)['win_amount'][['user_type'== "BOT"]]
    # loss_amount_bot =  get_win_loss(closed)['lose_amount'][['user_type'== "BOT"]]

In [12]:
type(win_amount_user)

numpy.float64

In [17]:
import plotly.graph_objects as go


# Create figure and add traces
fig = go.Figure()
fig.add_trace(go.Bar(
    x=df.index,
    y=df['win_amount'],
    name='Win',
    marker=dict(color='blue')
))
fig.add_trace(go.Bar(
    x=df.index,
    y=df['lose_amount'],
    name='Loss',
    marker=dict(color='red')
))
# Update layout
fig.update_layout(
    barmode='group',  # To stack bars on top of each other
    xaxis=dict(title='Timeframe'),
    yaxis=dict(title='Amount ($)'),
    title='Loss and Earn Amounts Over Timeframe',
    legend=dict(title='Category')
)

# Show figure
fig.show()


### Apply json type


In [6]:
datafield = ['open.json','close.json']

#make a function to read json
with open(datafield[1], 'r') as f:
    close = json.load(f)

def get_close_attribute(close):
    df = pd.DataFrame(close)
    df = df[['direction','margin_amount','stop_loss_give','take_profit_take']]
    df.groupby('direction').sum()
    return df

    

[{'user_type': 'USER', 'pair': 'ORAI_USDC', 'hash': 'AA677938E124AE43C10D99B1B205D3717B11AAC2B6555BBF4800EB622DC796B3', 'height': 14291688, 'time': '2023-11-08 10:20:57', 'order_id': 6, 'status': 'Close', 'direction': 'BUY', 'bidder_addr': 'orai12zyu8w93h0q2lcnt50g3fn0w3yqnhy4fvawaqz', 'offer_amount': 2.996398, 'ask_amount': 0.76634, 'leverage': 1.499099, 'margin_amount': 1.9988, 'entry_price': 3.910013, 'take_profit': 4.040346, 'stop_loss': 0, 'pnl': -2e-06, 'fee': 0.0012, 'funding_payment': 0, '': '', '__1': ''}, {'user_type': 'USER', 'pair': 'ORAI_USDC', 'hash': 'DBF51D9CC4C6DDC2AFE1C1BBE5157361919B5B7752216ED4DDD6D708F5179ED9', 'height': 14291669, 'time': '2023-11-08 10:19:09', 'order_id': 7, 'status': 'Close', 'direction': 'BUY', 'bidder_addr': 'orai12zyu8w93h0q2lcnt50g3fn0w3yqnhy4fvawaqz', 'offer_amount': 0.999598, 'ask_amount': 0.25565, 'leverage': 1, 'margin_amount': 0.9996, 'entry_price': 3.910033, 'take_profit': 5.865049, 'stop_loss': 0, 'pnl': -2e-06, 'fee': 0.0004, 'funding